군집화

In [ ]:
from sklearn.datasets import load_iris

data = load_iris()

X = data.data
y = data.target

print(data.target_names)

아래 코드는 분류와 군집화의 차이를 보여주는 그림을 그린다.
- 왼쪽
    - 각 샘플이 속하는 품종에 따른 분류
- 오른쪽
    - 지정된 기준에 따라 비슷한 샘플들끼리 그룹 짓기

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9, 3.5))
# 왼편 그림: 분류
plt.subplot(121)
plt.plot(X[y==0, 2], X[y==0, 3], "yo", label="Iris setosa")
plt.plot(X[y==1, 2], X[y==1, 3], "bs", label="Iris versicolor")
plt.plot(X[y==2, 2], X[y==2, 3], "g^", label="Iris virginica")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(fontsize=12)

# 오른편 그림: 군집화
plt.subplot(122)
plt.scatter(X[:, 2], X[:, 3], c="k", marker=".")
plt.xlabel("Petal length", fontsize=14)
plt.tick_params(labelleft=False)

plt.show()

꽃잎 길이와 너비만으로는 두 개의 군집으로만 구분이 가능해 보인다.
하지만, 꽃잎의 길이와 너비와 더불어, 꽃받침의 길이와 너비까지 포함한 4개의 특성을 모두 사용하여 후반부에서 가우시안 혼합 모델을 이용하여 세 개의 군집으로 나눌 수 있다.

In [ ]:
from sklearn.mixture import GaussianMixture

y_pred = GaussianMixture(n_components=3, random_state=42).fit(X).predict(X)

실제 타깃을 이용한 분류와 직접 비교하면 아래와 같으며, 버시컬러와 버지니카 품종의 군집이 거의 정확하게 구분된 것을 확인할 수 있다.

In [ ]:
plt.figure(figsize=(9, 3.5))

# 왼편 그림: 실제 품종별 분류
plt.subplot(121)
plt.plot(X[y==0, 2], X[y==0, 3], "yo", label="Iris setosa")
plt.plot(X[y==1, 2], X[y==1, 3], "bs", label="Iris versicolor")
plt.plot(X[y==2, 2], X[y==2, 3], "g^", label="Iris virginica")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(fontsize=12)

# 오른편 그림: 3개의 군집
plt.subplot(122)
plt.plot(X[y_pred==2, 2], X[y_pred==2, 3], "yo", label="Cluster 1")   # 2번 군집: 세토사
plt.plot(X[y_pred==0, 2], X[y_pred==0, 3], "bs", label="Cluster 2")   # 0번 군집: 버시컬러
plt.plot(X[y_pred==1, 2], X[y_pred==1, 3], "g^", label="Cluster 3")   # 1번 군집: 버지니카
plt.xlabel("Petal length", fontsize=14)
plt.legend(loc="upper left", fontsize=12)
plt.tick_params(labelleft=False)

plt.show()

군집화의 정확도를 확인하기 위해 군집별로 가장 많이 포함된 품종 즉, 품종의 최빈값을 확인한다.

아래 코드는 사이파이의 통계 모듈에 포함되어 있는 mode() 함수를 사용하여 각 군집별 최빈값을 확인한 후, 해당 최빈값과 군집 인덱스를 연결한다.

In [ ]:
from scipy import stats
import numpy as np

mapping = {}

for class_id in np.unique(y):                   # 품종 아이디: 0, 1, 2
    mode, _ = stats.mode(y_pred[y==class_id])   # mode: 지정된 품종이 가장 많이 포함된 군집 인덱스
    mapping[mode] = class_id                 # 군집 인덱스와 품종 연결

최종 결과는 아래와 같다.
- 1번 인덱스 군집
    - 세토사(0)가 제일 많다.
- 2번 인덱스 군집
    - 버시컬러(1)가 제일 많다.
- 0번 인덱스 군집
    - 버지니카(2)가 제일 많다.

In [ ]:
print(mapping)

mapping을 이용하여 군집 인덱스를 품종 인덱스로 변경한 후, 군집화의 정확도를 측정하면 96.7%가 나온다.

In [ ]:
y_pred = np.array([mapping[cluster_id] for cluster_id in y_pred])

np.sum(y_pred==y) / len(y_pred)

K-평균

먼저 2000개의 데이터 샘플을 생성한다.
생성되는 데이터는 지정된 5개의 센터를 중심으로 지정된 표준편차를 따르는 원 모양의 데이터 군집을 이룬다.
또한, 각각의 군집은 거의 동일한 크기를 갖는다.

5개의 센터와 표준편차

In [ ]:
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])

blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

make_blobs()함수가 앞서 설명한 방식으로 데이터를 생성한다.
- 각 샘플에 대한 실제 군집 인덱스가 타깃으로 함께 제공되지만, 여기서는 사용하지 않는다.

In [ ]:
from sklearn.datasets import make_blobs

X, _ = make_blobs(n_samples=2000, centers=blob_centers, cluster_std=blob_std, random_state=7)

산점도를 그리면 아래와 같다.

In [ ]:
def plot_clusters(X, y=None):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)

plt.figure(figsize=(8, 4))
plot_clusters(X)

plt.show()

군집화 훈련과 예측

사이킷런의 KMeans모델의 fit() 메서드는 각 군집의 중심(센트로이드)을 잡은 다음, 모든 샘플에 대해 가장 가까운 센트로이드를 중심으로 하는 군집을 이루도록 한다.
다만, 센트로이드를 몇 개로 할 것인지 먼저 지정해야 하며, 여기서는 5개를 사용하여 훈련시킨다.

In [ ]:
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

훈련 결과 kmeans.labels_ 속성에 각 훈련 샘플에 대한 군집 인덱스가 저장된다.

앞서 붓꽃 데이터 군집화에서 살펴본 것처럼 데이터가 생성될 때 지정된 실제 군집 인덱스가 아닌 훈련 과정에서 무작위로 지정된 인덱스임에 주의해야 한다.

In [ ]:
print(kmeans.labels_)

각 군집의 중심, 즉 5개의 센트로이드의 좌표는 다음과 같다.

In [ ]:
print(kmeans.cluster_centers_)

predict() 메서드를 사용하여 새로운 데이터에 대한 군집 예측도 가능하다.

In [ ]:
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])

print(kmeans.predict(X_new))

결정경계

군집을 나누는 결정경계를 그려보면, 보로노이 다이어그램이 생성된다.

plot_data() 함수는 여기서만 사용되는 기본값이 지정된 산점도를 그린다.

In [ ]:
# 산점도 그리기
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], "k.", markersize=2)

plot_centroids() 함수는 센트로이드를 시각화한다.
- weights=None 옵션
    - 특정 가중치 이상의 센트로이드만 그리도록 하는 설정이다.
    - 나중에 가우시안 혼합 모델을 시각화할 때 사용한다.

In [ ]:
# 센트로이드 그리기
def plot_centroids(centroids, weights=None, circle_color="w", cross_color="k"):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]

        plt.scatter(centroids[:, 0], centroids[:, 1], marker="0", s=35, linewidths=8, color=circle_color, zorder=10, alpha=0.9)

        plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

plot_decision_boundaries() 함수는 결정경계를 시각화한다.
- clusterer: 훈련된 군집화 모델 객체
- X: 훈련 세트

In [ ]:
def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    
    # 바탕화면 그리기
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    
    # 훈련 샘플 산점도 그리기
    plot_data(X)
    
    # 센트로이드 그리기
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    # 기타: x, y 축 레이블
    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
plt.figure(figsize=(8, 4))

plot_decision_boundaries(kmeans, X)

plt.show()

경계 근처의 일부 데이터가 잘못된 군집에 포함되긴 했지만, 전반적으로 군집이 잘 형성되었다.

군집화와 차원축소

지금까지 살펴 보았듯이 k-평균 모델 객체의 labels_속성은 각 샘플에 대해 가장 가까운 센트로이드를 중심으로 하는 군집의 인덱스를 지정하며, 이를 이용하여 predict() 메서드는 샘플이 속하는 군집의 인덱스를 반환한다.
- 이러한 방식의 군집화가 하드 군집화(hardd clustering)이다.

반면, 소프트 군집화(soft clustering)는 샘플과 각 군집 사이의 관계를 점수로 부여한다.
점수는 예를 들어 각 군집과 샘플사이의 거리 또는 5장에서 소개한 가우시안 방사기저 함수를 이용한 유사도 점수 등이 사용될 수 있다.
k-평균 모델 객체의 transform() 메서드는 샘플과 각 센트로이드 사이의 (유클리드) 거리를 점수로 사용한다.

아래 코드는 4개의 새 데이터를 변환하는 것을 보여준다.

In [ ]:
kmeans.transform(X_new)

앞서 설명한 대로 각 샘플에 대한 반환값은 각 센트로이드로부터 (유클리드) 거리임을 아래와 같이 확인할 수 있다.
- np.tile()
    - 주어진 어레이를 타일 모양의 지정된 형식으로 복제해서 이어붙이는 함수이다.

In [ ]:
np.linalg.norm(np.tile(X_new, (1, k)).reshape(-1, k, 2) - kmeans.cluster_centers_, axis=2)

k-평균 모델의 transform() 메서드는 결론적으로 기존 n-차원의 데이터셋을 비선형적으로 k-차원의 데이터셋으로 변환한는 비선형 차원축소 기법으로 사용될 수 있다.
- 국소적 선형 임베딩처럼 차원축소 기법을 군집화에도 활용할 수 있다.


K-평균 알고리즘

K-평균 알고리즘은 가장 빠르며, 가장 단순한 군집화 알고리즘 중 하나이다.
- k개의 센트로이드 무작위 지정
- 센트로이드의 위치가 특정 위치에 수렴할 때까지 아래 과정을 반복한다.
    - 모든 샘플에 대해 가장 가까운 센트로이드를 지정한다.
    - 각각의 센트로이드와 연결된 샘플들의 평균으로 해당 센트로이드 위치를 수정한다.

사이킷런의 KMeans 클래스

Kmeans 클래스의 기본 옵션 중 몇 가지는 다음과 같다.
- init="k-means++": 초기화 알고리즘
    - "random"을 사용할 경우 무작위 지정
    - "k-means++": 센트로이드 간의 거리를 최대한 크게 하는 알고리즘이다. 기본값으로 사용된다.
- n_init=10 : 센트로이드 반복 지정 횟수이다. 기본값은 10이다.
- algorithm="elkan": k-평균 알고리즘
    - "full": 앞서 설명한 알고리즘
    - "elkan" : 개선된 알고리즘
    - "auto": 기본적으로 "elkan"을 선택한다.
- max_iter=300: 초기화된 센트로이드 위치 반복 업데이트 횟수이다.

여기서는 예시를 위해 아래 옵션을 대신 사용한다.
- init="random"
- n_init=1
- algorithm="full"
- max_iter: 1, 2, 3

In [ ]:
kmeans_iter1 = KMeans(n_clusters=5, init="random", n_init=1, algorithm="full", max_iter=1, random_state=0)
kmeans_iter2 = KMeans(n_clusters=5, init="random", n_init=1, algorithm="full", max_iter=2, random_state=0)
kmeans_iter3 = KMeans(n_clusters=5, init="random", n_init=1, algorithm="full", max_iter=3, random_state=0)

kmeans_iter1.fit(X)
kmeans_iter2.fit(X)
kmeans_iter3.fit(X)

각 단계별 결정경계와 센트로이드의 변화는 아래와 같다.

In [ ]:
plt.figure(figsize=(10, 8))

# 맨 위 왼편
plt.subplot(321)
plot_data(X)
plot_centroids(kmeans_iter1.cluster_centers_, circle_color='r', cross_color='w')
plt.ylabel("$x_2$", fontsize=14, rotation=0)
plt.tick_params(labelbottom=False)
plt.title("Update the centroids (initially randomly)", fontsize=14)

# 맨 위 오른편
plt.subplot(322)
plot_decision_boundaries(kmeans_iter1, X, show_xlabels=False, show_ylabels=False)
plt.title("Label the instances", fontsize=14)

# 가운데 왼편
plt.subplot(323)
plot_decision_boundaries(kmeans_iter1, X, show_centroids=False, show_xlabels=False)
plot_centroids(kmeans_iter2.cluster_centers_)

# 가운데 오른편
plt.subplot(324)
plot_decision_boundaries(kmeans_iter2, X, show_xlabels=False, show_ylabels=False)

# 맨 아래 왼편
plt.subplot(325)
plot_decision_boundaries(kmeans_iter2, X, show_centroids=False)
plot_centroids(kmeans_iter3.cluster_centers_)

# 맨 아래 오른편
plt.subplot(326)
plot_decision_boundaries(kmeans_iter3, X, show_ylabels=False)

plt.show()

센트로이드 초기화 문제와 해결법

초기화 문제

초기화가 무작위로 이뤄질경우 적절하지 않은 군집화를 얻을 수 있다.
아래 코드는 2개의 나쁜 경우를 잘 보여준다.
- plot_clusterer_comparison() 함수는 두 개의 결정경계를 그래프를 동시에 그려준다.

In [ ]:
def plot_clusterer_comparison(cluster1, cluster2, X, title1=None, title2=None):
    cluster1.fit(X)
    cluster2.fit(X)

    plt.figure(figsize=(10, 3.2))

    plt.subplot(121)
    plot_decision_boundaries(cluster1, X)
    if title1:
        plt.title(title1, fontsize=14)

    plt.subplot(122)
    plot_decision_boundaries(cluster2, X, show_ylabels=False)
    if title2:
        plt.title(title2, fontsize=14)

아래 두 개의 그래프는 바로 이전의 경우처럼 센트로이드 초기화를 무작위로 딱 한 번만 사용한 결과를 보여준다.
- 두 경우 모두 적절하지 않은 모델을 생성한다.

In [ ]:
kmeans_rnd_init1 = KMeans(n_clusters=5, init="random", n_init=1, algorithm="full", random_state=2)
kmeans_rnd_init2 = KMeans(n_clusters=5, init="random", n_init=1, algorithm="full", random_state=5)

plot_clusterer_comparison(kmeans_rnd_init1, kmeans_rnd_init2, X, "Solution 1", "Solution 2 (with a different random init)")

plt.show()

관성(inertia: 이니셔)

비지도 학습인 k-평균 모델의 성능을 관성(inertia)을 이용하여 측정한다.
- 관성: 각 훈련 샘플과 가장 가까운 센트로이드 사이의 거리를 모두 합한 값이다.
    - inertia_ 속성에 저장된다.

In [ ]:
print(kmeans.inertia_)

아래 코드는 관성이 각 훈련 샘플과 가장 가까운 센트로이드 사이의 거리를 모두 합한 값임을 확인해준다.

In [ ]:
X_dist = kmeans.transform(X) # 각 샘플과 센트로이드들 사이의 거리이다.

np.sum(X_dist[np.arange(len(X_dist)), kmeans.labels_]**2) # 팬시 인덱싱 활용

score() 메서드는 관성의 음수값을 반환한다.
- 이유는 "더 높은 점수가 더 좋다"는 원칙을 따라야 하기 때문이다.

In [ ]:
print(kmeans.score(X))

초기화 반복

무작위 초기화 문제를 해결하기 위해 k-평균 알고리즘의 초기화를 여러 번 실행한 다음, 가장 낮은 관성을 보이는 모델을 최종 모델로 선택하며, 실제로 이 옵션(n_init=10)이 앞서 설명한 대로 KMeans 모델의 기본 하이퍼파라미터 값으로 설정되어 있다.

아래에서 확인할 수 있듯이 기본 하이퍼파라미터를 사용한 kmeans의 관성이 한 번의 센트로이드 초기화를 사용하는 모델들의 관성보다 낮다.

In [ ]:
print(kmeans_rnd_init1.inertia_)
print(kmeans_rnd_init2.inertia_)

실제로 n_init=10로 설정할 경우, 앞서 살펴본 좋은 모델과 비슷한 결과를 얻는다.

In [ ]:
kmeans_rnd_10_inits = KMeans(n_clusters=5, init="random", n_init=10, algorithm="full",random_state=2)
kmeans_rnd_10_inits.fit(X)

In [ ]:
plt.figure(figsize=(8, 4))
plot_decision_boundaries(kmeans_rnd_10_inits, X)
plt.show()

관성 점수도 거의 최저이다.

In [ ]:
print(kmeans_rnd_10_inits.inertia_)

K-Means++ 알고리즘

확률 계산으로 인해 초기화 비용이 좀 더 많이 들어가긴 하지만, 결과적으로 초기화 함수(n_init)를 획기적으로 줄일 수 있는 장점이 크다.
- 사이킷런의 KMeans 모델의 기본값으로 사용ㅎ나다.

데이터를 생성할 때 사용된 군집의 실제 중심과 kmeans 모델이 찾아낸 군집의 센트로이드가 매우 비슷함을 아래 코드에서 확인할 수 있다.

In [ ]:
print(blob_centers)

print(kmeans.cluster_centers_)

init 하이퍼파라미터 활용

센트로이드에 대한 좋은 후보값을 알 수 있다면 init 하이퍼파라미터의 값으로 센트로이드의 좌표를 지정하여 훈련하면 보다 좋은 결과를 얻게됨을 아래 코드에서 볼 수 있다.
- 아래 5개의 좌표가 엎서 언급한 blob_centers의 좌표와 유사하다.
    - 여기서 순서는 중요하지 않다.

In [ ]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])

kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42)
kmeans.fit(X)
kmeans.inertia_

개선된 K-평균 알고리즘

사이킷런의 KMeans 모델은 algorithm="elkan"을 기본 옵션으로 사용한다.
그런데, elkan 알고리즘은 희소 데이터에 대해서는 잘 작동하지 않기에, 모든 거리를 계산하는 algorithm="full" 옵션이 희소 데이터에 대해 자동으로 선택된다.

아래 코드는 두 방식의 시간 차이를 보여준다.
- 하지만, 데이터셋이 크지 않기에 시간 차이가 크지 않아 보인다.

In [ ]:
%timeit -n 50 KMeans(algorithm="elkan", random_state=42).fit(X)

In [ ]:
%timeit -n 50 KMeans(algorithm="full", random_state=42).fit(X)

미니배치 K-평균

사이킷런의 MiniBatchKMeans 모델은 미니배치 학습을 지원한다.
- batch_size=100: 배치 크기 지정, 기본값은 100

In [ ]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5, random_state=42)
minibatch_kmeans.fit(X)

In [ ]:
print(minibatch_kmeans.inertia_)

훈련 세트가 많이 큰 경우: memmap 클래스 활용

MNIST 데이터셋을 대상으로 미니배치 k-평균 모델을 훈련한다.

In [ ]:
import urllib.request
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1)
mnist.target = mnist.target.astype(np.int64) # 타깃의 자료형을 변환해줘야한다.

훈련 세트와 테스트 세트로 구분한다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist["data"], mnist["target"], random_state=42)

memmap 객체로 지정한다.

In [ ]:
filename="my_mnist.data"
X_mm = np.memmap(filename, dtype="float32", mode="write", shape=X_train.shape)
X_mm[:] = X_train

MiniBatchKMeans 모델을 훈련한다.

In [ ]:
minibatch_kmeans = MiniBatchKMeans(n_clusters=10, batch_size=10, random_state=42)
minibatch_kmeans.fit(X_mm)

훈련 세트가 너무 큰 경우: partial_fit() 활용

데이터셋이 너무 크다면 memmap 클래스조차 활용하지 못할 수 있다.
이럴때는 메모리가 아닌 다른 저장 장치로부터 필요한 만큼의 데이터 배치(묶음)을 불러오는 함수를 이용하여 수동으로 미니배치 학습을 구현해야 한다.
- 즉, 아래 내용을 직접 구현해야 한다.

- 초기화 반복
- 최적 모델 기억하기, 즉 최소 관성값을 갖는 모델 기억하기

아래 함수는 지정된 크기 만큼의 데이터를 무작위로 선택하여 전달한다.

In [ ]:
def load_next_batch(batch_size):
    return X[np.random.choice(len(X), batch_size, replace=False)]

다음 조건에 맞게 미니배치 모델을 훈련한다.

In [ ]:
k = 5 # 센트로이드 개수
n_init = 10 # 센트로이드 초기화 횟수
n_iterations = 100 # 센트로이드 조정 횟수
batch_size = 100 # 배치 크기
init_size = 500 # K-평균++ 알고리즘의 초기화 후보에 사용될 데이터셋 크기

아래 코드는 초기화를 반복하면서 최적의 모델을 업데이트한다.
- partial_fit() 메서드를 사용함에 주의해야 한다.

In [ ]:
np.random.seed(42)

evaluate_on_last_n_iters = 10 # 센트로이드 조정 마지막 10단계 모델의 관성 누적합 저장 기준
best_kmeans = None # 최고 모델 저장

for init in range(n_init): # 초기화 반복
    # 미니배치 k-평균 모델 초기화 및 partial_fit() 훈련
    minibatch_kmeans = MiniBatchKMeans(n_clusters=k, init_size=init_size)
    X_init = load_next_batch(init_size)
    minibatch_kmeans.partial_fit(X_init)

    # 센트로이드 조정마지막 10단계 모델의 관성 누적합 저장
    minibatch_kmeans.sum_inertia_ = 0

    # 센트로이드 조정
    for iteration in range(n_iterations):
        X_batch = load_next_batch(batch_size)
        minibatch_kmeans.partial_fit(X_batch)

        # 누적 관성 계산
        if iteration >= n_iterations - evaluate_on_last_n_iters:
            minibatch_kmeans.sum_inertia_ += minibatch_kmeans.inertia_
    # 최저 누적 관성 모델 업데이트
            if (best_kmeans is None or minibatch_kmeans.sum_inertia_ < best_kmeans.sum_inertia_):
                best_kmeans = minibatch_kmeans

        

훈련된 모델의 점수는 매우 높은 편이다.

In [ ]:
print(best_kmeans.score(X))

미니배치 K-평균 알고리즘이 일반 K-평균 알고리즘보다 훨씬 빠르다.

In [ ]:
%timeit KMeans(n_clusters=5, random_state=42).fit(X)

In [ ]:
%timeit MiniBatchKMeans(n_clusters=5, random_state=42).fit(X)

반면, 성능은 많이 떨어진다. 이 사실을 확인해야 할 군집수와 관련해서 확인한다.
- 먼저 군집수를 1에서 100까지 변화시키면서 일반 K-평균 모델과 미니배치 K-평균 모델이 훈련에 필요한 시간과 훈련된 모델의 관성을 측정한다.

In [ ]:
from timeit import timeit

times = np.empty((100, 2))
inertias = np.empty((100, 2))

for k in range(1, 101):
    kmeans_ = KMeans(n_clusters=k, random_state=42)
    minibatch_kmeans = MiniBatchKMeans(n_clusters=k, random_state=42)

    print("\r{} / {}".format(k, 100), end="")
    times[k-1, 0] = timeit("kmeans_.fit(X)", number=10, globals=globals())
    times[k-1, 1]  = timeit("minibatch_kmeans.fit(X)", number=10, globals=globals())
    inertias[k-1, 0] = kmeans_.inertia_
    inertias[k-1, 1] = minibatch_kmeans.inertia_

군집수를 늘릴 때 훈련시간과 관성의 변화를 그래프로 그리면 다음과 같다.
- 왼쪽 그림
    - 관성
- 오른쪽 그림
    - 훈련 속도

In [ ]:
plt.figure(figsize=(10,4))

# 왼편 그림
plt.subplot(121)

plt.plot(range(1, 101), inertias[:, 0], "r--", label="K-Means")            # 빨강 파선
plt.plot(range(1, 101), inertias[:, 1], "b.-", label="Mini-batch K-Means") # 파랑 실선

plt.xlabel("$k$", fontsize=16)
plt.title("Inertia", fontsize=14)
plt.legend(fontsize=14)
plt.axis([1, 100, 0, 100])

# 오른편 그림
plt.subplot(122)

plt.plot(range(1, 101), times[:, 0], "r--", label="K-Means")            # 빨강 파선
plt.plot(range(1, 101), times[:, 1], "b.-", label="Mini-batch K-Means") # 파랑 실선

plt.xlabel("$k$", fontsize=16)
plt.title("Training time (seconds)", fontsize=14)
plt.axis([1, 100, 0, 6])

plt.show()

최적의 군집수

지금까지 사용한 예제에 대한 군집수를 5보다 작거나 크게 하면 아래와 같은 일이 발생한다.

In [ ]:
kmeans_k3 = KMeans(n_clusters=3, random_state=42)  # 3개의 군집
kmeans_k8 = KMeans(n_clusters=8, random_state=42)  # 8개의 군집

plot_clusterer_comparison(kmeans_k3, kmeans_k8, X, "$k=3$", "$k=8$")
plt.show()

관성 활용

별로 좋아보이지 않는다.
그런데, 관성은 군집수가 커질수록 줄어든다.

In [ ]:
print(kmeans_k3.inertia_)
print(kmeans_k8.inertia_)

실제로 군집수가 많아질 수록 관성은 줄어든다.
- 이유는 센트로이드 수가 늘어날 수록 각 샘플과 센트로이드 사이의 거리는 줄어들 수 밖에 없기 때문이다.
- 아래 코드가 이 사실을 잘 보여준다.

- k가 1부터 9까지 변하는 동안 훈련된 모델의 관성을 측정한다.

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X) for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

- 군집수와 측정된 관성 사이의 관계를 선그래프로 그리면 아래와 같다.

In [ ]:
plt.figure(figsize=(8, 3.5))

# 군집수와 관성 관계
plt.plot(range(1, 10), inertias, "bo-")

plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)

# 주석 작성: Elbow 단어와 화살표 표시
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )

plt.axis([1, 8.5, 0, 1300])

plt.show()

Elbow에 해당하는 위치인 k=4 즉, 4개의 군집이 좋아 보인다.
군집이 4개보다 작으면 별로이고, 4개보다 많아도 별로 좋아지지 않아 보인다.

In [ ]:
plot_decision_boundaries(kmeans_per_k[4 - 1], X)
plt.show()

실루엣 점수

실루엣 점수(Silhouete Score)는 각 훈련 샘플에 대한 실루엣 계수(Silhouette Coefficient)의 평균값이다.

실루엣 계수는 -1과 1 사이의 값이다.
- 1에 가까울 때
    - 해당 인스턴스가 속하는 군집의 한 센트로이드에 가깝게 위치한다.
- 0에 가까울 때
    - 해당 인스턴스가 속하는 군집의 경계에 가깝게 위치한다.
- -1에 가까울 때
    - 해당 인스턴스가 속하는 군집이 아닌 다른 군집의 센트로이드에 가깝게 위치한다.
    - 즉, 잘못된 군집에 속한다.

아래 코드는 군집수가 증가할 때 실루엣 점수의 변화를 보여준다.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = [
    silhouette_score(X, model.labels_) for model in kmeans_per_k[1:]
]

plt.figure(figsize=(8, 3))
plt.plot(range(2, 10), silhouette_scores, "bo-")

plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([1.8, 8.5, 0.55, 0.7])

plt.show()

k=4가 여전히 매우 좋아 보인다.
하지만, 관성의 경우와는 달리 k=5도 역시 좋다는 것을 알 수 있다.

군집별로 각 샘플의 실루엣 계수를 오름차순으로 정렬한 그래프인 실루엣 다이어그램이 보다 많은 정보를 전달한다.

In [ ]:

from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter
import matplotlib as mpl

plt.figure(figsize=(11, 9))

for k in (3, 4, 5, 6):
    plt.subplot(2, 2, k - 2)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = mpl.cm.Spectral(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)

plt.show()

모든 군집이 거의 비슷한 크기이고, 모든 군집의 칼날이 실루엣 점수(빨강 파선)을 넘어가고 있다면 좋은 모델로 평가할 수 있다.

K-평균의 한계

K-평균의 가장 큰 단점은 최적의 군집수를 확인하기 위해 알고리즘을 여러 번 실행해야 한다는 점이다.
또한, 군집의 크기와 밀도가 서로 다르거나, 원형이 아닌 경우 K-평균 모델이 제대로 작동하지 않을 수 있다.

In [ ]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1=X1.dot(np.array([[0.384, 0.95], [0.732, 0.598]]))

X2, y2 = make_blobs(n_samples=250, centers = 1, random_state=42)
X2 = X2 + [6, -8]

X = np.r_[X1, X2]
y = np.r_[y1, y2]

plot_clusters(X)

먼저 알고 있는 센트로이드 정보를 이용하여 좋은 K-평균 모델을 훈련한다.

In [ ]:
kmeans_good = KMeans(n_clusters=3, init=np.array([[-1.5, 2.5], [0.5, 0], [4, 0]]), n_init=1, random_state=42)
kmeans_good.fit(X)

이후, 센트로이드를 무작위로 지정한다.

In [ ]:
kmeans_bad = KMeans(n_clusters=3, random_state=42)
kmeans_bad.fit(X)

두 모델의 훈련 결과는 아래와 같다.
오른편 모델은 잘 훈련되지 않았다.
왼편 모델은 25%의 데이터가 오른쪽 군집에 잘못 할당되었다.

In [ ]:
plt.figure(figsize=(10, 3.2))

plt.subplot(121)
plot_decision_boundaries(kmeans_good, X)
plt.title("Inertia = {:.1f}".format(kmeans_good.inertia_), fontsize=14)

plt.subplot(122)
plot_decision_boundaries(kmeans_bad, X, show_ylabels=False)
plt.title("Inertia = {:.1f}".format(kmeans_bad.inertia_), fontsize=14)

plt.show()

특성 스케일링 활용

특성 스케일링을 하면 군집 구분이 보다 명확해지고 보다 원형에 가까운 군집이 생성되어 K-평균 모델의 성능이 보다 좋아질 수 있다.

군집화 활용: 이미지 색상 분할

이미지 색상 분할은 유사한 색상을 동일한 군집에 연결하는 기법이다.
색상 분할 과정을 살펴보기 위해 무당벌래 이미지 하나를 다운로드 한다.

In [ ]:
import os
# 무당벌레 이미지 다운로드
PROJECT_ROOT_DIR = "."
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"

filename = "ladybug.png"
print("Downloading", filename)

url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

다운로드 된 이미지는 533 x 800 픽셀 크기의 칼라 사진이다.

In [ ]:
from matplotlib.image import imread
image = imread(os.path.join(images_path, filename))

image.shape

K-평균 모델 훈련을 위해 이미지 픽셀을 1차원으로 변환한다.
- 즉, (533, 800, 3) 모양의 어레이를 (426400, 3) 모양의 어레이로 변환한다.

In [ ]:
X = image.reshape(-1, 3)
X.shape

이미지 색상 분할 과정

아래 코드는 8개의 군집을 이용한 색상 분할 과정을 보여준다.
먼저, k-평균 모델을 설정한다.

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)

찾아낸 8개의 센트로이드는 다음과 같다.

In [ ]:
print(kmeans.cluster_centers_)

각 훈련 샘플에 대한 레이블(kmeans.labels_)을 이용하여 군집별 센트로이드의 색상으로 통일시킨다.
이를 위해 넘파이 어레이에 대한 팬시 인덱싱을 활용한다.

In [ ]:
segmented_img = kmeans.cluster_centers_[kmeans.labels_]

이미지를 다시 원래 크기로 변환한다.

In [ ]:
segmented_img = segmented_img.reshape(image.shape)

군집수에 따른 비교

아래 코드는 앞서 설명한 방식을 다양한 군집수에 대해 적용한 결과를 비교할 수 있는 그림을 보여준다.

- 5종류의 군집수를 이용한 이미지 색상 분할

In [ ]:
segmented_imgs = []
n_colors = (10, 8, 6, 4, 2)

for n_clusters in n_colors:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    segmented_img = kmeans.cluster_centers_[kmeans.labels_]
    segmented_imgs.append(segmented_img.reshape(image.shape))

- 원본 이미지와 군집화를 활용한 5개의 이미지 비교
    - 군집수가 적어질수록 작은 무당벌레의 색상을 다른 군집과 연결하기에 무당벌레 색상이 점차 사라진다.

In [ ]:
plt.figure(figsize=(10,5))
plt.subplots_adjust(wspace=0.05, hspace=0.1)

# 원본 이미지
plt.subplot(231)
plt.imshow(image)
plt.title("Original image")
plt.axis('off')

# 색상 분할된 이미지 5개
for idx, n_clusters in enumerate(n_colors):
    plt.subplot(232 + idx)
    plt.imshow(segmented_imgs[idx])
    plt.title("{} colors".format(n_clusters))
    plt.axis('off')

plt.show()

군집화 활용: 차원 축소

훈련된 k-평균 객체의 transform() 메서드는 주어진 데이터에 대하여 각 센트로이드부터의 거리로 이뤄진 어레이를 생성한다.
즉, n 차원의 데이터셋을 k차원의 데이터셋으로 변환한다.
만약 k < n 이라면, 이는 비선형 차원축소로 간주될 수 있으며, 이어지는 지도학습에 유용하게 활용될 수 있다.

여기서는 MNIST 손글씨 데이터셋을 이용하여 군집화를 차원축소 기법으로 활용하는 방식을 소개한다.
다만, 여기서 사용되는 데이터셋은 8 X 8 크기의 픽셀을 갖는 1797개의 축소된 MNIST 데이터셋을 사용한다.

In [ ]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

훈련 세트와 테스트 세트로 구분한다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=42)

로지스틱 회귀 적용

로지스틱 회귀 모델을 훈련한 뒤, 다음 테스트 세트에 대한 성능을 평가하면, 정확도 평균값이 96.8888% 정도로 나온다.

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train)

log_reg.score(X_test, y_test)

군집화 전처리 후 성능 평가

아래 파이프라인은 50개의 군집으로 먼저 군집화를 한 뒤, 로지스틱 회귀 모델을 훈련한다.

파이프라인으로 구성된 예측기의 fit() 메서드를 호출하면, 최종 예측기를 제외한 나머지 예측기의 fit_transform() 메서드가 호출된다.
또한, K-평균 모델의 transform() 메서드는 앞서 설명한 방식으로 차원축소를 진행한다.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("log_reg", LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)),
])
pipeline.fit(X_train, y_train)

파이프라인 모델의 정확도 평균값은 97.7%로 28%의 성능향상이 발생한다.

In [ ]:
print(pipeline.score(X_test, y_test))

print(
1 - (1 - 0.977777) / (1 - 0.968888))

그리드 탐색 활용

그리드 탐색을 이용하여 최적의 군집수를알아낼 수 있다.
즉, 파이프라인의 성능이 최고가 되도록 하는 군집수는 다음과 같다.

아래 코드는 그리드 탐색을 이용하여 2개부터 100개 사이의 군집을 사용할 때 이어지는 로지스틱 회귀의 성능을 평가하여 최적의 군집수를 알아낸다.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

최적의 군집수는 57이며, 로지스틱 회귀의 정확도는 98%이다.

In [ ]:
print(grid_clf.best_params_)
print(grid_clf.score(X_test, y_test))

군집화 활용: 준지도학습

준지도 학습은 약간의 레이블이 있는 샘플이 있고, 대부분의 샘플에는 레이블이 없는 데이터셋에 대한 지도학습 기법이다.
주지도핛브 설명을 위해 미니 MNIST 데이터셋을 계속 이용한다.

먼저, 무작위로 선정된 50개의 샘플만을 대상으로 로지스틱 회귀모델을 훈련하면 정확도 평균값이 83.33% 정도로 낮게 나온다.
이유는 훈련 세트가 작아서 훈련이 제대로 되지 않기 때문이다.

In [ ]:
n_labeled = 50

log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", random_state=42)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

log_reg.score(X_test, y_test)

대표이미지 활용

무작위로 50개의 샘플을 선택해서 훈련하는 대신, 50개의 군집으로 군집화한 뒤 각 군집의 센트로이드에 가장 가까운 이미지 50개를 대상으로 훈련해보았을 때 로지스틱 회귀 모델의 성능을 확인한다.

센트로이드에 가장 가까운 이미지를 대표이미지라 부른다.

먼저, 50개의 군집으로 나눈 후 변환한다.

In [ ]:
k = 50

kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)

변환된 훈련세트는 50개의 특성을 가지며, 샘플멸로 50개 군집의 센트로이드 사이의 거리를 특성값으로 갖는다.
따라서, 특성별로 최소값을 갖는 인덱스가 50개 군집의 센트로이드에 가장 가까운 샘플을 가리킨다.
이 성질을 이용하여 대표이미지를 아래와 같이 선정한다.

In [ ]:
representative_digit_idx = np.argmin(X_digits_dist, axis=0) # 50개의 대표이미지 인덱스 확인
X_representative_digits = X_train[representative_digit_idx] # 50개의 대표이미지 지정

선정된 50개의 대표이미지에 포함된 숫자는 다음과 같다.

In [ ]:
plt.figure(figsize=(8, 2))

for index, X_representative_digit in enumerate(X_representative_digits):
    plt.subplot(k // 10, 10, index + 1)
    plt.imshow(X_representative_digit.reshape(8, 8), cmap="binary", interpolation="bilinear")
    plt.axis('off')

plt.show()

실제 타깃(레이블)은 다음과 같다.

In [ ]:
print(y_train[representative_digit_idx])

위 정보를 이용하여 50개의 대표이미지에 대한 레이블을 지정한다.

In [ ]:
y_representative_digits = y_train[representative_digit_idx]

50개의 대표이미지를 이용하여 로지스틱 회귀 모델을 훈련시킨 결과 성능이 89.5% 이상으로 좋아졌다.
이를 통해 무작위로 선정된 샘플의 레이블을 이용하는 것보다 대표(샘플)을 군집화를 이용하여 선정한 후 학습하면 보다 좋은 성능의 모델이 생성됨을 확인할 수 있다.

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

레이블 전파 1

동일한 군지벵 속하는 샘플의 레이블을 대표이미지의 레이블로 지정하는 레이블 전파 방식을 활용할 때의 성능을 확인한다.

아래 코드는 군집별로 샘플의 레이블을 대표이미지의 레이블로 지정한다.

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)

for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

로지스틱 회귀 모델의 훈련 후 성능은 92.8% 정도로 조금 향상된다.

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

레이블 전파 2

군집 전체에 레이블을 전파하면 이상치 등에 대한 레이블 전파도 이뤄지기에 모델의 성능을 약화시킬 수 있다.
따라서 군집별로 센트로이드에 가까운 20%의 샘플에 대해서만 레이블을 전파하고, 나머지 샘플은 무시한다.

아래 코드는 각 샘플에 대해 해당 샘플이 속한 군집 센트로이드까지의 거리를 확인한다.

In [ ]:
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]

아래 코드는 군집별로 센트로이드 근접도가 상위 20%가 아닌 샘플들을 센트로이드와의 거리를 -1로 지정하는 방식을 이용하여 제외시킨다.

In [ ]:
percentile_closest = 20

for i in range(k):
    in_cluster = (kmeans.labels_ == i) # 군집별 샘플 대상
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest) # 군집별 센트로이드 근접도 상위 2
    above_cutoff = (X_cluster_dist > cutoff_distance) # 군집별 센트로이드 근접도 상위 20% 이내 샘플 
    X_cluster_dist[in_cluster & above_cutoff] = -1

아래 코드는 군집별로 센트로이드 근접도가 상위 20%안에 드는 샘플만 훈련세트로 추출한다.

In [ ]:
partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

선정된 샘플만을 이용하여 로지스틱 회귀 모델을 훈련한 후 성능을 평가한다.

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)

log_reg.score(X_test, y_test)

레이블 전파 정확도

센트로이드 근접도가 상위 20% 이내인 샘플들을 대상으로 전파된 레이블의 정확도는 98.2%에 달한다.

In [ ]:

np.mean(y_train_partially_propagated == y_train[partially_propagated])

반면, 전체 훈련 세트에 대해 레이블 전파를 했을 경우 정확도는 93.5% 정도이다.

In [ ]:
np.mean(y_train_propagated == y_train)

DBSCAN

DBSCAN 알고리즘은 데이터셋의 밀도를 이용하여 군집을 구성하며, 하나의 군집은 연속된 핵심 샘플(core samples)들의 엡실론 이웃을 모두 합한 결과이다.

핵심 샘플인지 여부는 반경 엡실론과 min_samples 두 개의 하이퍼파라미터에 의해 결정되며, 하나의 핵심 샘플이 다른 핵심 샘플의 엡실론 반경 안에 들어갈 때 연속된 핵심 샘플이라 한다.
따라서 하나의 군집은 핵심 샘플의 이웃의 이웃 관계로 이뤄진 샘플들의 집합니다.

DBSCAN 알고리즘은 모든 군집이 충분한 밀도의 샘플로 구성되고 군집 사이는 저밀도 지역으로 구분될 수 있을 때 잘 작동한다.
아래 예제는 moons 데이터셋을 이용하여 DBSCAN 알고리즘의 작동과정을 보여준다.
- 1000개의 샘플로 이뤄진 moons 데이터셋을 생성한다.

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

DBSCAN 모델 훈련
- eps = 0.05: 엡실론 반경
- min_samples: epsilon 반경 안에 있어야 하는 샘플의 수

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

총 7개의 군집이 형성되었다.

In [ ]:
np.unique(dbscan.labels_)

일부 샘플의 군집 인덱스가 -1인 이유는 해당 샘플들이 이상치로 간주되었음을 의미한다.

In [ ]:
dbscan.labels_[:10]

핵심 샘플은 총 808개로 지정되었다.

In [ ]:
len(dbscan.core_sample_indices_)

청므 10개의 핵심 샘플의 인덱스는 다음과 같다.

In [ ]:
dbscan.core_sample_indices_[:10]

핵심 샘플 자체는 components_ 속성이 기억한다. 처음 3개의 핵심 샘플은 다음과 같다.

In [ ]:
dbscan.components_[:3]

이제 이웃의 반경을 0.2로 키워보자

In [ ]:
dbscan2 = DBSCAN(eps=0.2)
dbscan2.fit(X)

그러면 군집수가 2로 줄어들고, 이상치도 사라진다,

In [ ]:
np.unique(dbscan2.labels_)

plot_dbscan() 함수는 군집화된 샘플을 보여준다.
군집은 색으로 구분되며 각 샘플을 핵심 샘플, 이상치, 기타로 구분한다.

핵심 샘플
- 지정된 반경 안에 지정된 수의 샘플이 포함되는 샘플이다.
- 군집별 다른 색상을 사용한다.

이상치
- 어떤 핵심 샘플의 반경 안에도 포함되어 있지 않은 샘플이다.
- 빨강 X로 표시한다.

기타
- 다른 핵심 샘플의 반경 안에 있지만 자체는 핵심 샘플이 아닌 경우 이다.
- 검은 점으로 표시한다.

In [ ]:
def plot_dbscan(dbscan, X, size, show_xlabels=True, show_ylabels=True):
    
    # 코어 샘플 구분용 마스크 설정
    core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
    core_mask[dbscan.core_sample_indices_] = True
    
    # 이상치 구분용 마스크 설정
    anomalies_mask = dbscan.labels_ == -1
    
    # 기타 샘플용 마스크 설정
    non_core_mask = ~(core_mask | anomalies_mask)

    # 핵심 샘플
    cores = dbscan.components_
    # 이상치
    anomalies = X[anomalies_mask]
    # 기타
    non_cores = X[non_core_mask]
    
    # 핵심 샘플 산점도: 각 샘플과 함께 반경도 함께 표현
    plt.scatter(cores[:, 0], cores[:, 1],
                c=dbscan.labels_[core_mask], marker='o', s=size, cmap="Paired")
    plt.scatter(cores[:, 0], cores[:, 1], marker='*', s=20, c=dbscan.labels_[core_mask])
    
    # 이상치 산점도: 빨강 X 표시
    plt.scatter(anomalies[:, 0], anomalies[:, 1], c="r", marker="x", s=100)
    
    # 기타 샘플 산점도: 책과는 달리 명확한 구분을 위해 검은 점으로 표시
#     plt.scatter(non_cores[:, 0], non_cores[:, 1], c=dbscan.labels_[non_core_mask], marker=".")
    plt.scatter(non_cores[:, 0], non_cores[:, 1], c="k", marker=".")
    
    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
    
    plt.title("eps={:.2f}, min_samples={}".format(dbscan.eps, dbscan.min_samples), fontsize=14)

아래 코드는 위 두 경우를 비교하는 그림을 그린다.
오른편 그림에서는 기타의 경우는 존재하지 않는다.
즉, 모든 샘플이 핵심 샘플이다.
반면, 왼편 그림에선 3종류의 샘플이 표시된다.

핵심 샘플: 808개
이상치: 77개
기타: 115개

In [ ]:
# 핵심 샘플 수
len(dbscan.components_)

In [ ]:
# 이상치 수
(dbscan.labels_==-1).sum()

In [ ]:
plt.figure(figsize=(9, 3.2))

# 왼편 그림: 반경 0.05
plt.subplot(121)
plot_dbscan(dbscan, X, size=100)

# 오른편 그림: 반경 0.2
plt.subplot(122)
plot_dbscan(dbscan2, X, size=600, show_ylabels=False)

plt.show()

DBSCAN과 예측하기

DBSCAN 모델을 predict() 메서드를 지원하지 않으며, fit_predict() 메서드만 지원한다.
- DBSCAN 모델의 특성상 예측을 하려면 예측에 사용되는 데이터 샘플을 포함하여 핵심 샘플을 다시 계산해야 하기 때문인데, 이는 예측할 때마다 매 번 학습을 다시 해야 한다는 것을 의미한다.

반면, 주어진 샘플의 가장 가까운 군집을 예측하는 일은 간단하게 구현된다.
예를 들어, K-최근접 이웃 분류기인 KNeighborsClassifier 모델을 훈련하기만 하면 된다.
이 방식을 설명하기 위해 앞서 훈련한 dbscan2 모델(위 그림의 오른편 모델)을 사용한다.

In [ ]:
dbscan = dbscan2

K-최근접 이웃 분류기

K-최근접 이웃 분류 모델은 주어진 샘플 근처의 K개의 샘플에 대한 타깃값들의 최빈값(mode)을 주어진 샘플의 예측값으로 지정한다.

아래 코드는 학습된 DBSCAN 모델의 핵심 샘플만을 이용하여 k-최근접 이웃 분류기를 훈련한다.
이웃 샘플의 수는 50으로 지정한다.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

훈련된 k-최근접 이웃 분류기를 이용하여 새로운 샘플들에 대한 클래스를 예측한다.

In [ ]:
X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
knn.predict(X_new)

k-최근접 이웃 분류기는 각 클래스에 속할 확률도 계산한다.
아래 결과에서 볼 수 있듯이 predict_proba() 메서드는 0번과 1번 클래스에 속할 확률을 각 샘플에 대해 계산한다.

In [ ]:
knn.predict_proba(X_new)

아래 코드는 4개의 새로운 샘플에 대한 예측값을 포함하여 두 군집의 결정경계를 잘 보여준다.

In [ ]:
plt.figure(figsize=(6, 3))

# 결정경계 그리기: knn 모델의 predict() 메서드 활용
plot_decision_boundaries(knn, X, show_centroids=False)

# 네 개의 샘플 표기: 파랑 + 기호
plt.scatter(X_new[:, 0], X_new[:, 1], c="b", marker="+", s=200, zorder=10)

plt.show()

이상치 처리

앞서 사용된 훈련 세트에서는 이상치가 없어서 모든 샘플에 대한 군집(클래스)을 할당한다.
하지만, 모든 군집으로부터 일정 거리 이상 떨어진 샘플을 간단하게 이상치로 처리할 수 있다.

아래 코드는 k-최근접 이웃 분류기의 kneighbors() 메서드를 이용하여 가장 가까운 이웃 샘플로부터의 거리를 계산한다.
- kneighbors() 메서드는 n_neighbors로 지정된 수 만큼의 가장 가까운 데이터들의 인덱스와 거리로 이뤄진 두 개의 어레이를 반환한다.

In [ ]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[y_pred_idx]   # 가장 가까운 샘플의 (클래스) 레이블
y_pred[y_dist > 0.2] = -1             # 거리가 0.2 이상인 경우 이상치 처리
y_pred.ravel()

계층적 DBSCAN: HDBSCAN

DBSCAN 모델은 군집의 밀집도가 변하는 데이터셋에서는 잘 작동하지 않는다.
이러한 한계를 극복하기 위해 다양한 엡실론을 이용하여 최적의 군집을 찾는 HDBSCAN 모델을 활용할 수 있다.

In [ ]:
# 훈련 세트
data, _ = make_blobs(1000, centers=5)

# hdbscan 모델 훈련
from hdbscan import HDBSCAN

clusterer = HDBSCAN(min_cluster_size=10, gen_min_span_tree=True)
clusterer.fit(data)

# 생성된 군집
np.unique(clusterer.labels_)

기타 군집화 알고리즘

스펙트럼 군집화(Spectral Clustering)

샘플 사이의 유사도 행렬을 이용하여 저차원으로 차원축소를 진행한 후, k-평균 등의 군집화를 진행하는 기법이다.
유사도 행렬은 가우시안 rbf(방사기저함수) 등을 이용하여 계산하며, 차원축소는 8장에서 소개한 커널 PCA와 유사한 방식으로 이뤄진다.

사이킷런의 SpectralClustering 모델은 군집수를 지정해야 하며, 간단한 사용법은 다음과 같다.

In [ ]:
from sklearn.cluster import SpectralClustering

gamma는 8장에서 소개한 rbf 함수에 사용되는 감마 계수에 해당한다.
gamma 값이 커질 수록 보다 가까운 샘플들 사이에서의 유사도가 강조된다.

아래 코드는 유사도를 달리한 군집화의 결과를 보여준다.

In [ ]:
sc1 = SpectralClustering(n_clusters=2, gamma=100, random_state=42)
sc1.fit(X)

In [ ]:
sc2 = SpectralClustering(n_clusters=2, gamma=1, random_state=42)
sc2.fit(X)

- 군집화 결과를 보여주는 함수

In [ ]:
def plot_spectral_clustering(sc, X, size, alpha, show_xlabels=True, show_ylabels=True):
    plt.scatter(X[:, 0], X[:, 1], marker='o', s=size, c='gray', cmap="Paired", alpha=alpha)
    plt.scatter(X[:, 0], X[:, 1], marker='o', s=30, c='w')
    plt.scatter(X[:, 0], X[:, 1], marker='.', s=10, c=sc.labels_, cmap="Paired")
    
    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
        
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
    
    plt.title("RBF gamma={}".format(sc.gamma), fontsize=14)

오른편 그래프의 경우 gamma=1에 해당하며, 잘못된 군집화를 보여준다.
반면, 왼편 그래프는 gamma=100에 해당하며 제대로된 군집화를 보여준다.

In [ ]:
plt.figure(figsize=(9, 3.2))

plt.subplot(121)
plot_spectral_clustering(sc1, X, size=500, alpha=0.1)

plt.subplot(122)
plot_spectral_clustering(sc2, X, size=4000, alpha=0.01, show_ylabels=False)

plt.show()

병합 군집화(Agglomerative Clustering)

병합 군집화는 계층 군집화의 주요 기법이다.
가장 작은 단위의 군집은 개별 샘플 하나로 이뤄지며, 두 개의 군집을 합치며 점점 군집의 수를 줄여 나간다.
군집화 대상인 두 개의 군집은 병합했을 때 두 샘플 사이의 연결 거리가 최소가 되도록 유도하며, 이를 위해 탐욕 알고리즘을 사용한다.

아래 코드는 사이킷런의 AgglomerativeClustering 클래스를 이용하여 병합 군집화 모델을 훈련하는 과정을 보여준다.
연결 거리는 다음 옵션 종류에 따라 다르게 측정되며, 기본 옵션은 "ward"이다.
- linkage: "ward", "complete", "average", "single"

In [ ]:
from sklearn.cluster import AgglomerativeClustering

X = np.array([0, 2, 5, 8.5]).reshape(-1, 1)

print(X)

agg = AgglomerativeClustering(linkage="complete").fit(X)

아래 함수는 훈련된 군집화 모델이 제공하는 속성의 리스트를 제공한다.

In [ ]:
def learned_parameters(estimator):
    return [attrib for attrib in dir(estimator)
            if attrib.endswith("_") and not attrib.startswith("_")]

병합 군집 모델의 경우 아래 속성이 제공된다.

In [ ]:
learned_parameters(agg)

예를 들어, children_ 속성은 병합과정을 묘사하는 트리를 (m, 1, 2) 모양의 어레이로 설명한다.
어레이에 사용된 숫자 i의 의미는 다음과 같다.

- i< m 인 경우
    - i번째 샘플이며, 리프 노드에 해당한다.
- i >= m인 경우
    - i - m번 인덱스에 포함된 항목들을 자식 노드로 가지는 노드이다.

In [ ]:
agg.children_

위 결과의 의미는 다음과 같다.
- 0, 1, 2, 3: 주어진 4개의 샘플
- 4: children_의 0번 인덱스에 위치한 [0, 1]에 포함된 0번, 1번 노드를 자식 노드로 갖는 노드
- 5: children_의 1번 인덱스에 위치한 [2, 3]에 포함된 2번, 3번 노드를 자식 노드로 갖는 노드
- 6: children_의 2번 인덱스에 위치한 [4, 5]에 포함된 4번, 5번 노드를 자식 노드로 갖는 노드이자, 루트 노드 즉, 전체 샘플로 구성된 한 개의 클래스를 나타낸다.

두 개의 군집을 생성했기에, 4번 5번 노드가 각각 서로 다른 군집을 가리킨다.

In [ ]:
agg.labels_

가우시안 혼합(Gaussian Mxitures)

가우시안 혼합 모델은 데이터셋이 여러 개의 가우시안 분포가 혼합된 분포를 따른다고 가정하는 확률 모델이다.
하나의 가우시안 분포를 따르는 샘플들이 하나의 군집을 생성하며, 일반적으로 타원형 모습을 띈다.
타원의 모양, 크기, 밀집도, 방향이 다양한 여러 개의 가우시안 분포를 따르는 데이터 셋을 군집화한다.

아래 코드는 가우시안 혼합 모델을 지원하는 사이킷런의 GaussianMixture 클래스의 활용법을 설명하기 위해 사용되는 데이터셋을 생성한다.
- 군집 수: 3개
- 군집 크기 비 : 500 : 500 : 250 즉, 4 : 4 : 2
- X1: 두 개의 군집으로 이뤄진 데이터셋
- X2: 다른 군집에 비해 밀도가 낮다.

In [ ]:
# 군집 2개
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))

# 셋째 군집
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]

# 데이터셋 병합
X = np.r_[X1, X2]
y = np.r_[y1, y2]

GaussianMixture 모델을 적용한다.

In [ ]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

가중치(weights_)는 군집별 크기 비를 나타내며, 거의 제대로 학습되었다.

In [ ]:
gm.weights_

학습된 군집별 평균값과 공분산은 다음과 같다.

In [ ]:
print(gm.means_)
print("----------------")
print(gm.covariances_)

기대값-최적화(EM: Expectation-Maximization) 알고리즘이 4번 반복만에 수렴된 것을 확인할 수 있다.

In [ ]:
print(gm.converged_)
print(gm.n_iter_)

predict()와 predict_proba()
- predict() 메서드: 군집 인덱스를 직접 지정한다.
- predict_proba() 메서드: 군집별 속할 확률

In [ ]:
print(gm.predict(X))
print(gm.predict_proba(X))

생성모델

생성모델은 학습된 정보를 이용하여 새로운 샘플을 생성할 수 있으며, 속한 군집에 대한 정보도 함께 제공한다.

In [ ]:
X_new, y_new = gm.sample(6)

print(X_new)

군집 인덱스 순서대로 샘플을 생성한다.

In [ ]:
print(y_new)

생성된 샘플들의 군집별 비는 군집별 가중치를 따른다.

In [ ]:
from collections import Counter

X_new ,y_new = gm.sample(1000)
Counter(y_new)

score_samples() 와 확률밀도

샘플별로 확률밀도함수(PDF: Probability Density Function)의 로그를 계산한다.

In [ ]:
print(gm.score_samples(X))

In [ ]:
print(np.exp(gm.score_samples(X_new)))

아래 코드는 확률밀도함수와 x축으로 감싸인 면적이 1임을 증명한다.
증명 방식은 구분구적법(Segmentation Method)을 이용한 정적분 계산이다.

먼저 x축과 y축 모두 -10부터 10 사이의 구간을 1/100 단위로 구분하여 가로세로 20인 정사각형 영역을 2000x2000개의 작은 격자로 잘게 쪼갠다.
-10부터 10사이의 구간을 택한 이유는 해당 영역 이외에서는 확률밀도가 거의 0이기 때문이다.

In [ ]:
resolution = 100
grid = np.arange(-10, 10, 1 / resolution)
xx, yy = np.meshgrid(grid, grid)

X_full = np.vstack([xx.ravel(), yy.ravel()]).T

이제 각 격자(정사각형)의 한쪽 모서리에서의 확률밀도를 해당 격자의 면적과 곱한 값을 모두 더하면 거의 1이다.
- score_samples() 메서드의 반환값은 확률밀도의 로그임에 주의해야한다.
- 격자의 한 변의 길이는 1/100이다.

In [ ]:
# exp() 함수를 적용하여 log() 함수를 상쇄시킴
pdf = np.exp(gm.score_samples(X_full))

# 격자의 크기를 확률밀도와 곱하기
pdf_probas = pdf * (1 / resolution) ** 2
pdf_probas.sum()

결정경계와 밀도 등고선

plot_gaussian_mixture() 함수는 가우시안 혼합 모델이 알아낸 군집 결정경계와 샘플의 로그밀도 등고선(density contours)을 그린다.

In [ ]:
from matplotlib.colors import LogNorm

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    # 로그밀도 등고선 그리기
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    # score_samples가 기본적으로 음수이기에 양수로 변환함
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    # 결정경계 그리기: 빨강 파선
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z, linewidths=2, colors='r', linestyles='dashed')
    
    # 데이터 산점도 및 센트로이드 그리기
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

군집 경계와 로그밀도 등고선이 제대로 그려진다.

In [ ]:
plt.figure(figsize=(8, 4))

plot_gaussian_mixture(gm, X)

plt.show()

covariance_type 속성과 군집 형태

군집에 속하는 데이터들의 가우시안 분포를 결정하는 공분산 행렬은 공분산 유형(covariance_type) 옵션에 따라 달라진다.
- "full": 어떤 제한도 가하지 않는다. 즉, 군집은 임의의 크기와 모양의 타원 형태를 가질 수 있다.
- "tied": 모든 군집이 동일한 모양과 크기의 타원 형태이어야 한다.
- "spherical": 모든 군집이 원 형태를 가져야 한다. 지름은 다를 수 있다.
- "diag": 모든 군집이 임의의 크기와 모양의 타원 형태를 가져야 한다. 단, 타원의 축이 기본 축과 평행해야 한다. 즉, 공분산 행렬이 대각 행렬이어야 한다.

아래 코드는 다양한 방식으로 가우시안 혼합 모델을 훈련시킨다.

In [ ]:
gm_full = GaussianMixture(n_components=3, n_init=10, covariance_type="full", random_state=42)
gm_tied = GaussianMixture(n_components=3, n_init=10, covariance_type="tied", random_state=42)
gm_spherical = GaussianMixture(n_components=3, n_init=10, covariance_type="spherical", random_state=42)
gm_diag = GaussianMixture(n_components=3, n_init=10, covariance_type="diag", random_state=42)
gm_full.fit(X)
gm_tied.fit(X)
gm_spherical.fit(X)
gm_diag.fit(X)

compare_gaussian_mixtures() 함수는 서로 다른 옵션을 사용하는 두 모델의 결과를 비교하는 그래프를 그린다.

In [ ]:
def compare_gaussian_mixtures(gm1, gm2, X):
    plt.figure(figsize=(9, 4))

    plt.subplot(121)
    plot_gaussian_mixture(gm1, X)
    plt.title('covariance_type="{}"'.format(gm1.covariance_type), fontsize=14)

    plt.subplot(122)
    plot_gaussian_mixture(gm2, X, show_ylabels=False)
    plt.title('covariance_type="{}"'.format(gm2.covariance_type), fontsize=14)

"tide" 방식과 "spherical" 방식 사이의 차이는 다음과 같다.

In [ ]:
compare_gaussian_mixtures(gm_tied, gm_spherical, X)

plt.show()

"full" 방식과 "diag" 방식 사이의 차이는 다음과 같다.

In [ ]:
compare_gaussian_mixtures(gm_full, gm_diag, X)
plt.tight_layout()
plt.show()

가우시안 혼합과 이상치 탐지

지정된 밀도보다 낮은 저밀도 지역에 위치한 샘플을 이상치로 간주할 수 있다.
밀도 기준은 경우에 따라 다르게 지정된다.
예를 들어, 아래 코드는 제조 결함이 4% 정도라고 알려져 있다면 밀도 임계값을 4%의 제조 결함률이 이상치로 판명되도록 정한다.

In [ ]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)  # 4%를 이상치로 처리하는 밀도 임곗값
anomalies = X[densities < density_threshold]     # 이상치

아래 코드는 밀도 4% 이하의 지역에 위치한 이상치를 표시한다.

In [ ]:
plt.figure(figsize=(8, 4))

plot_gaussian_mixture(gm, X)
plt.scatter(anomalies[:, 0], anomalies[:, 1], color='r', marker='*')  # 이상치 표시: 빨강 별표
plt.ylim(top=5.1)

plt.show()

군집수 선택

k-평균 모델의 경우와는 달리 군집이 임의의 타원 형태를 띌 수 있기에 적절한 군집수를 선택하기 위해 관성(이너셔), 실루엣 점수 등을 활용할 수 없다.
대신, BIC 또는 AIC와 같은 이론적 정보 기준을 사용할 수 있다.

- m: 훈련 세트 크기
- p: 모델이 학습해야 하는 파라미터 개수
- L_hat: 모델의 가능도 함수(likelihood function)의 최대값

BIC(베이즈 정보 기준: Bayesian Information Criterion)와 AIC(아카이케 정보 기준: Akaike Information Criterion) 모두 값이 낮을 수록 좋은 모델이다.
이는 모델이 학습해야 하는 파라미터가 적을 수록 불리함을 의미하며, 결국 군집수를 적게하는 방향으로 유도한다.
또한, 모델의 가능도 함수의 값이 클 수록 좋으며 따라서 데이터를 보다 잘 대표하는 모델일 수록 두 기준값이 낮아진다.

학습된 모델의 BIC와 AIC는 각각 bic(), aic() 메서드가 계산한다.

In [ ]:
print(gm.bic(X))
print(gm.aic(X))

BIC와 AIC

모델이 학습해야 하는 파라미터는 아래 3종류이다.
- 군집별 가중치
    - 군집별로 하나의 가중치가 필요하지만, 가중치의 합이 1이어야 하기에 (군집수 - 1)이 사용된다.
- 군집별 평균값
    - (군집수 * 차원)을 사용한다.
- 군집별 공분산
    - (군집수 * (1+2+...+차원))

n x n 공분산 행렬의 자유도(파라미터 수)는 n^2이 아니라 1 + 2 + ... + n = n(n + 1) / 2 이다.

In [ ]:
# p = (군집수 - 1) + (군집수 * 차원) + (군집수 * 차원 * (차원+1) // 2)
n_clusters = 3
n_dims = 2
n_params_for_weights = n_clusters - 1      # 군집별 가중치
n_params_for_means = n_clusters * n_dims   # 군집별 평균값
n_params_for_covariance = n_clusters * n_dims * (n_dims + 1) // 2   # 군집별 공분산
n_params = n_params_for_weights + n_params_for_means + n_params_for_covariance

# 각 샘플에 대한 최대 로그 가능도를 모두 더한 값 
max_log_likelihood = gm.score(X) * len(X)

bic = np.log(len(X)) * n_params - 2 * max_log_likelihood
aic = 2 * n_params - 2 * max_log_likelihood

In [ ]:
print(bic, aic)

군집수에 따른 BIC와 AIC

군집수를 달리하면서 BIC와 AIC의 변화를 관측한다.
아래 코드는 군집수를 1개에서 10개까지 변화시키면서 10개의 가우시안 혼합 모델을 훈련한다.

In [ ]:
gms_per_k = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(X) for k in range(1, 11)]

bics = [model.bic(X) for model in gms_per_k]
aics = [model.aic(X) for model in gms_per_k]

훈련된 모델의 BIC와 AIC의 변화를그래프로 그리면 다음과 같다.

In [ ]:
plt.figure(figsize=(8, 3))

plt.plot(range(1, 11), bics, "bo-", label="BIC")
plt.plot(range(1, 11), aics, "go--", label="AIC")

plt.xlabel("$k$", fontsize=14)
plt.ylabel("Information Criterion", fontsize=14)
plt.axis([1, 9.5, np.min(aics) - 50, np.max(aics) + 50])

plt.annotate('Minimum',
             xy=(3, bics[2]),
             xytext=(0.35, 0.6),
             textcoords='figure fraction',
             fontsize=14,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )

plt.legend()

plt.show()

아래 코드는 군집수뿐만 아니라, 공분산 유형(covariance_type)을 달리하면서 가장 좋은 BIC를 갖는 가우시안 혼합 모델을 찾는다.

In [ ]:
min_bic = np.infty

for k in range(1, 11):
    for covariance_type in ("full", "tied", "spherical", "diag"):
        bic = GaussianMixture(n_components=k, n_init=10,
                              covariance_type=covariance_type,
                              random_state=42).fit(X).bic(X)
        if bic < min_bic:
            min_bic = bic
            best_k = k
            best_covariance_type = covariance_type

3개의 군집과 full 공분산 유형이 가장 좋은 모델을 생성한다.

In [ ]:
print(best_k)

print(best_covariance_type)

가능도(likelihood) 함수

가능도 함수는 파라미터가 알려지지 않은 분포에서 특정 확률변수 값 x가 주어졌을 때 파라미터의 변화에 따라 해당 값이 나타날 확률을 계산한다.
반면, 최대 가능도 추정치(MLE: Maximal Likelihood Estimate)는 주어진 확률변수 값이 발생할 확률을 최대로 하는 확률분포의 파라미터이다.

가능도 함수의 최대값을 찾는 대신, 가능도 함수값의 로그값을 최대화시키는 작업을 사용하곤 한다.
이유는 두 함수값을 최대화 하는 파라미터가 동일한 반면, 함수의 로그값의 최대값을 구하는 일이 경우에 따라 훨신 쉬워지기 떄문이다.

아래 코드는 두 개의 가우시안 분포가 혼합된 확률분포 모델을 생성한다.
- 두 가우시안 분포 중심: -4와 1
- 적절한 스케일 적용, 앞서 사용한 구분구적법을 이용하면, 정적분값이 1이 되도록 스케일링한다.

In [ ]:
from scipy.stats import norm

# x축 구간: -6에서 4.
xx = np.linspace(-6, 4, 101)
# y축 구간: 1에서 2
ss = np.linspace(1, 2, 101)

# 지정된 구간을 100x100 개의 격자로 쪼갬
XX, SS = np.meshgrid(xx, ss)

# 확률밀도 함숫(pdf)값 지정: 적절한 스케일 사용.
ZZ = 2 * norm.pdf(XX, 1, SS) + norm.pdf(XX, -4, SS)
ZZ = ZZ / ZZ.sum(axis=1)[:,np.newaxis] / (xx[1] - xx[0])

아래 코드는 338쪽의 그림 9-20을 그린다.

- 좌상단 그림
    - 모델의 파라미터 함수
- 우상단 그림
    - x=2.5일 때의 가능도 함수
- 좌하단 그림
    - 세타=1.3일 때의 확률밀도함수(PDF)
- 우하단 그림
    - x=2.5일 때의 로그 가능도 함수

In [ ]:
from matplotlib.patches import Polygon

plt.figure(figsize=(8, 4.5))

x_idx = 85   # x=2.5의 인덱스
s_idx = 30   # theta=1.3의 인덱스

# 좌상단 그림: 모델의 파라미터 함수 f(x;theta)
plt.subplot(221)
plt.contourf(XX, SS, ZZ, cmap="GnBu")
plt.plot([-6, 4], [ss[s_idx], ss[s_idx]], "k-", linewidth=2)
plt.plot([xx[x_idx], xx[x_idx]], [1, 2], "b-", linewidth=2)
plt.xlabel(r"$x$")
plt.ylabel(r"$\theta$", fontsize=14, rotation=0)
plt.title(r"Model $f(x; \theta)$", fontsize=14)

# 우상단 그림: x=2.5일 때의 가능도 함수 L(theta|x=2.5) = f(x=2.5;theta)
plt.subplot(222)
plt.plot(ss, ZZ[:, x_idx], "b-")
max_idx = np.argmax(ZZ[:, x_idx])     # MLE
max_val = np.max(ZZ[:, x_idx])
plt.plot(ss[max_idx], max_val, "r.")  # MLE 표시(빨강 점)
plt.plot([ss[max_idx], ss[max_idx]], [0, max_val], "r:")
plt.plot([0, ss[max_idx]], [max_val, max_val], "r:")
plt.text(1.01, max_val + 0.005, r"$\hat{L}$", fontsize=14)
plt.text(ss[max_idx]+ 0.01, 0.055, r"$\hat{\theta}$", fontsize=14)
plt.text(ss[max_idx]+ 0.01, max_val - 0.012, r"$Max$", fontsize=12)
plt.axis([1, 2, 0.05, 0.15])
plt.xlabel(r"$\theta$", fontsize=14)
plt.grid(True)
plt.text(1.99, 0.135, r"$=f(x=2.5; \theta)$", fontsize=14, ha="right")
plt.title(r"Likelihood function $\mathcal{L}(\theta|x=2.5)$", fontsize=14)

# 좌하단 그림: theta=1.3일 때의 확률밀도함수(pdf) f(x;theta=1.3)
plt.subplot(223)
plt.plot(xx, ZZ[s_idx], "k-")
plt.axis([-6, 4, 0, 0.25])
plt.xlabel(r"$x$", fontsize=14)
plt.grid(True)
plt.title(r"PDF $f(x; \theta=1.3)$", fontsize=14)
verts = [(xx[41], 0)] + list(zip(xx[41:81], ZZ[s_idx, 41:81])) + [(xx[80], 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
plt.gca().add_patch(poly)

# 우하단 그림: x=2.5일 때 로그 가능도 함수 log L(theta|x=2.5)
plt.subplot(224)
plt.plot(ss, np.log(ZZ[:, x_idx]), "b-")
max_idx = np.argmax(np.log(ZZ[:, x_idx]))
max_val = np.max(np.log(ZZ[:, x_idx]))
plt.plot(ss[max_idx], max_val, "r.")
plt.plot([ss[max_idx], ss[max_idx]], [-5, max_val], "r:")
plt.plot([0, ss[max_idx]], [max_val, max_val], "r:")
plt.axis([1, 2, -2.4, -2])
plt.xlabel(r"$\theta$", fontsize=14)
plt.text(ss[max_idx]+ 0.01, max_val - 0.05, r"$Max$", fontsize=12)
plt.text(ss[max_idx]+ 0.01, -2.39, r"$\hat{\theta}$", fontsize=14)
plt.text(1.01, max_val + 0.02, r"$\log \, \hat{L}$", fontsize=14)
plt.grid(True)
plt.title(r"$\log \, \mathcal{L}(\theta|x=2.5)$", fontsize=14)

plt.show()

베이즈 가우시안 혼합 모델

베이즈 가우시안 혼합 모델은 군집수를 미리 정하기 보다는 필요한 만큼의 군집만을 선택해서 사용한다.
사이킷런의 BayesianGaussianMixture 모델은 불필요한 군집에 0 또는 거의 0에 가까운 가중치(weights)를 가하는 방식으로 필요한 만큼의 군집만 사용한다.

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

모델을 생성할 때 군집수를 충분히 크다고 생각하는 수로 지정해야 한다.

In [ ]:
bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X)

모델이 적절한 군집화를 완료하지 못했다고 경고가 뜨는 경우에는 초기화 횟수(n_init) 또는 기대값-최대화(EM) 알고리즘이 반복 횟수(max_iter)를 키운다.

- 아래 코드는 max_iter를 기본값인 100 대신 150을 사용한다.

In [ ]:
bgm = BayesianGaussianMixture(n_components=10, n_init=10, max_iter=150, random_state=42)

bgm.fit(X)

아래 코드는 n_init를 15로 지정한다.

In [ ]:
bgm = BayesianGaussianMixture(n_components=10, n_init=15, random_state=42)
bgm.fit(X)

3개의 군집만 필요하다는 사실을 아래와 같이 확인한다.
또한, 군집 크기의 비가 4:4:2로 제대로 예측되었다.

In [ ]:
np.round(bgm.weights_, 2)

군집 결정경계는 다음과 같다.

In [ ]:
plt.figure(figsize=(8, 5))
plot_gaussian_mixture(bgm, X)
plt.show()

사전 확률

사전 믿음(prior belief) 즉, 군집수에 대한 사전 지식에 대한 믿음을 weight_concentration_prior 속성으로 지정할 수 있다.
0보다 큰 값을 사용하며, 0에 가까울수록 적은 군집수가 사용되었음을 암시한다.
지정하지 않으면 1/n_components가 기본값으로 사용된다.
하지만 훈련세트가 크면 별 의미가 없으며, 아래와 같은 그림은 사전 믿음의 차이가 매우 크고 훈련세트가 작은 경우에만 그려진다.


다음 코드는 사전확률을 극단적으로 차이를 두어 두 모델을 생성한다.
또한 73개의 샘플만을 이용하여 훈련한다.

In [ ]:
# 사전 믿음: 0.01
bgm_low = BayesianGaussianMixture(n_components=10, max_iter=1000, n_init=1,
                                  weight_concentration_prior=0.01, random_state=42)
# 사전 믿음: 10,000
bgm_high = BayesianGaussianMixture(n_components=10, max_iter=1000, n_init=1,
                                  weight_concentration_prior=10000, random_state=42)
# 훈련 세트 크기: 73
nn = 73
bgm_low.fit(X[:nn])
bgm_high.fit(X[:nn])

사전 믿음이 0.01인 경우 3 개의 군집만 찾는다.

In [ ]:
np.round(bgm_low.weights_, 2)

사전 믿음이 10,000인 경우 세, 네의 군집을 찾는다.

In [ ]:
np.round(bgm_high.weights_, 2)

In [ ]:
plt.figure(figsize=(9, 4))

# 왼편 그림
plt.subplot(121)
plot_gaussian_mixture(bgm_low, X[:nn])
plt.title("weight_concentration_prior = 0.01", fontsize=14)

# 오른편 그림
plt.subplot(122)
plot_gaussian_mixture(bgm_high, X[:nn], show_ylabels=False)
plt.title("weight_concentration_prior = 10000", fontsize=14)

plt.show()

실행 결과가 책과 다르다.
또한 훈련세트 크기를 조금만 다르게 조정해도 전혀 다른 결과를 얻는다.
이는 앞서 언급한 대로 사전 믿음이 주는 불안정성을 반영한 결과이다.

가우시안 혼합 모델의 한계

가우시안 혼합 모델은 데이터셋을 타원 형태의 군집으로 분할하려 한다.
따라서 moons 데이터셋과 같은 경우 제대로된 군집화를 진행하지 못한다.

In [ ]:
X_moons, y_moons = make_moons(n_samples=1000, noise=0.05, random_state=42)

bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X_moons)

아래 코드는 moons 데이터셋에 베이즈 가우시안 혼합 모델을 적용하여 8개의 군집을 찾는 것을 보여준다.
하지만, 밀도 고등선은 이상치 탐지에 활용되기에는 충분하다.

In [ ]:
plt.figure(figsize=(9, 3.2))

plt.subplot(121)
plot_data(X_moons)
plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14, rotation=0)

plt.subplot(122)
plot_gaussian_mixture(bgm, X_moons, show_ylabels=False)

plt.show()